In [50]:
using LsqFit

In [72]:
# From time to M
M(t::Array,t_p::Float64,P::Float64) = (2*pi/P).*(t.-t_p)
M(t::Float64,t_p::Float64,P::Float64) = (2*pi/P)*(t-t_p)

M (generic function with 2 methods)

In [89]:
# this should evaluate to zero when E is correct
g(E::Array,ecc,m::Array) = E .- ecc.*sin(E) .- m
g(E::Float64,ecc,m::Float64) = E - ecc*sin(E) - m

g (generic function with 3 methods)

In [90]:
# g'(E)
g_prime(E::Float64,ecc) = 1.0 - ecc*cos(E)
g_prime(E::Array,ecc) = 1.0 .- ecc.*cos(E)

g_prime (generic function with 2 methods)

In [164]:
#+1 for positive x, -1 for negative, with proper handling of zero division
function sign(x::Array)
    result = []
    for val in x
        if val!= 0.0
            push!(result,val/abs(val))
        else
            push!(result,0.0)
        end
    end
    return result
end
function sign(x::Float64)
    if x!= 0.0
        return x/abs(x)
    else
        return 0.0
    end
end

sign (generic function with 2 methods)

In [92]:
#Best guess at initial E0
E0(m::Array) = m .+ 0.85.*sign(sin(m))
E0(m::Float64) = m + 0.85*sign(sin(m))

E0 (generic function with 2 methods)

In [175]:
#Newtons takes a function, that function's derivative, a guess at the best value to make the 
#function equal to zero and the parameters to the function. delta can be specified depending
#on your machine's precision.

function newtons(f::Function,f_prime::Function,E_0::Float64,ecc::Float64,m::Float64;delta=1e-14)
    
    E_new = E_0
    dE = 1.
    
    while abs(dE)>delta
        E_old = E_new
        E_new = E_old - (f(E_old,ecc,m)/f_prime(E_old,ecc))
        dE = E_new-E_old
    end
    
    return E_new
    
end

function newtons(f::Function,f_prime::Function,E_0::Array,ecc::Float64,m::Array;delta=1e-14)
    
    E_news = []
    counter = 0
    for (val,em) in zip(E_0,m)
        counter+=1
        E_new = val
        dE = 1.0
        
        while abs(dE)>delta
            E_old = E_new
            E_new = E_old - (f(E_old,ecc,em)/f_prime(E_old,em))
            dE = E_new-E_old
        end
    
        push!(E_news,E_new)
    end
    
    return E_news
    
end

newtons (generic function with 2 methods)

In [184]:
#How to convert from E to f
EtoF(E::Float64,ecc::Float64) = 2.*atan((((1.+ecc)/(1.-ecc))^(1./2.))*tan(E/2.))
EtoF(E::Array,ecc::Float64) = 2.*atan((((1.+ecc)/(1.-ecc))^(1./2.)).*sin(E./2.)./cos(E./2.))

EtoF (generic function with 2 methods)

In [80]:
K(h::Float64,c::Float64) = sqrt(h*h+c*c)

K (generic function with 1 method)

In [81]:
pomega(h::Float64,c::Float64) = atand(-c/h)

pomega (generic function with 1 method)

In [95]:
gamma(v0::Float64,K::Float64,ecc::Float64,pomega::Float64) = v0-K*ecc*cosd(pomega)

gamma (generic function with 1 method)

In [83]:
function P_guesser(x_data::Array,y_data::Array)
    
    #minimum period to look for is distance between data points. Let's put the distances in an array
    x_dist=[]
    for i=2:size(x_data)[1]
        push!(x_dist,x_data[i]-x_data[i-1])
    end
    
    sq_array = []
    P_array = []
    
    for P=mean(x_dist):0.5:maximum(x_data)-minimum(y_data)
        
        phases = mod(x_data,P)
        RV_sort = y_data[sortperm(phases)] 
        
        sq = 0.0
        for i=2:size(RV_sort)[1]
            sq += (RV_sort[i]-RV_sort[i-1])^2.0
        end
        
        push!(sq_array,sq)
        push!(P_array,P)
    end
    
    return P_array[findmin(sq_array)[2]]
    
end

P_guesser (generic function with 1 method)

In [84]:
data = readdlm("mystery_planet.txt",Float64) ;

In [85]:
times = data[:,1]
RVs = data[:,2]
RV_errs = data[:,3] ;

In [190]:
function fit_RV(x_data,y_data,y_err)
    
    P_guess = P_guesser(x_data,y_data)
    
    #Because h, c and v0 depend on the data, we'll need to define the function when we fit
    
    function hcv0(f::Array)
        W = diagm(1.0./(y_err.^2.0))
        F = hcat(cos(f),sin(f),[1.0 for x in x_data])'
        epsilon = inv(F*W*F')
        return y_data'*W*F'*epsilon
    end
    function hcv0(ecc,P,t_p)
        W = diagm(1.0./(y_err.^2.0))
        F = hcat([cos(EtoF(newtons(g,g_prime,E0(M(t,t_p,P)),ecc,M(t,t_p,P)),ecc)) for t in x_data],[sin(EtoF(newtons(g,g_prime,E0(M(t,t_p,P)),ecc,M(t,t_p,P)),ecc)) for t in x_data],[1.0 for x in x_data])'
        epsilon = inv(F*W*F')
        return y_data'*W*F'*epsilon
    end
    
    function v_rad(t::Array,params) #params are e,P,t_p
        if 0 <= params[1] < 1
            f = EtoF(newtons(g,g_prime,E0(M(t,params[3],params[2])),params[1],M(t,params[3],params[2])),params[1])
            h,c,v0 = hcv0(f)
            return h.*cos(f)+c*sin(f).+v0
        else
            return Inf
        end
    end
    function v_rad(t::Float64,params)
        if 0.0 <= params[1] < 1.0
            f = EtoF(newtons(g,g_prime,E0(M(t,params[3],params[2])),params[1],M(t,params[3],params[2])),params[1])
            h,c,v0 = hcv0(params[1],params[2],params[3])
            return h*cos(f)+c*sin(f)+v0
        else
            return Inf
        end
    end

    
    p0 = [0.1,P_guess,x_data[1]]
    println(sin(newtons(g,g_prime,E0(M([x_data[1],x_data[2]],p0[3],p0[2])),p0[1],M([x_data[1],x_data[2]],p0[3],p0[2]))))
    
    #scatter(x_data,y_data,y_err)
    #scatter(x_data,v_rad(x_data,p0))
    
end

fit_RV (generic function with 1 method)

In [191]:
fit_RV(times,RVs,RV_errs)

LoadError: LoadError: MethodError: `sin` has no method matching sin(::Array{Any,1})
while loading In[191], in expression starting on line 1

In [189]:
methods(sin)

# 13 methods for generic function "sin":
sin(a::Complex{Float16}) at float16.jl:151
sin(z::Complex{T<:Real}) at complex.jl:548
sin(x::Float64) at math.jl:137
sin(x::Float32) at math.jl:138
sin(a::Float16) at float16.jl:150
sin(x::BigFloat) at mpfr.jl:610
sin(x::Real) at math.jl:139
sin{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) at sparse/sparsematrix.jl:648
sin{T<:Number}(::AbstractArray{T<:Number,1}) at operators.jl:380
sin{T<:Number}(::AbstractArray{T<:Number,2}) at operators.jl:381
sin{T<:Number}(::AbstractArray{T<:Number,N}) at operators.jl:383
sin(z::DualNumbers.Dual{T<:Union{Complex{T<:Real},Real}}) at /Users/trevorzaylen/.julia/v0.4/DualNumbers/src/dual.jl:271
sin(z::DualNumbers.Dual4{T<:Real}) at /Users/trevorzaylen/.julia/v0.4/DualNumbers/src/dual_n.jl:154